In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/FM_data/'
group_dir_ = '../data//FM_data/groups/random'
file_name = 'normalized_to_rating_filter_track_5_user_50.csv'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering(new)', str(0.005) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [3]:
predictions[:5]

,uid,tid,rating
9,0,48,2.318919
29,0,135,2.258065
34,0,176,2.323969
38,0,191,2.276103
69,0,354,2.247148


In [4]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [5]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [6]:
num_user = len(predictions['uid'].unique())
num_user

218

In [7]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [8]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [9]:
for i in precision_4:
    print(i)

for i in precision_20:
    print(i)

for i in precision_50:
    print(i)

for i in recall_4:
    print(i)

for i in recall_20:
    print(i)

for i in recall_50:
    print(i)

[0.16399082568807338, 0.15393518518518517, 0.13194444444444445, 0.13604651162790699, 0.12962962962962962, 0.1578341013824885, 0.12962962962962962, 0.12037037037037036, 0.10119047619047619, 0.10375]
[0.12133027522935781, 0.11944444444444449, 0.11226851851851855, 0.11627906976744186, 0.11574074074074081, 0.1188940092165899, 0.10532407407407414, 0.1011574074074075, 0.09952380952380963, 0.08625000000000003]
[0.1137614678899081, 0.10601851851851835, 0.10361111111111104, 0.10176744186046499, 0.09657407407407403, 0.09456221198156671, 0.09027777777777768, 0.0867592592592591, 0.0872380952380951, 0.0752999999999999]
[0.0015620363703718306, 0.001345399768653882, 0.0013732164348556493, 0.0012809078853279737, 0.0011739033009005996, 0.0013295313463843321, 0.00104232288736734, 0.0016601327735648176, 0.0006739487480098499, 0.0009762457501603369]
[0.006281834515166305, 0.006030076969385375, 0.005603120523066201, 0.005788747281965418, 0.005230013721391892, 0.006148913381275596, 0.004761676446423214, 0.0

In [10]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [11]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [12]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [13]:
for i in nDCG_4:
    print(i)

for i in nDCG_20:
    print(i)

for i in nDCG_50:
    print(i)

[0.08808934969481644, 0.07780436157614848, 0.051156344808308156, 0.07328502285898376, 0.06557391929967203, 0.07124408890211212, 0.051978695786182603, 0.05090356815501295, 0.049820225367131446, 0.0497066914486413]
[0.06313650643663163, 0.060374105161894426, 0.04698336876167795, 0.05651188743328302, 0.05405603786066622, 0.05293013753159295, 0.04472668839538717, 0.0456593582559448, 0.04285385527462993, 0.03665828031698409]
[0.05828793372457611, 0.05318878158930366, 0.046761679731669656, 0.051661962885899686, 0.04686828301475588, 0.04515660314253687, 0.043448283907839294, 0.04211575457176019, 0.04077789958595144, 0.03429867915497442]
